## import the libraries :

In [4]:
from tensorflow import keras 
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import  Conv2D, MaxPooling2D


# spliting of data into training and testing :
 
    
(x_train, y_train),(x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [19]:
import tensorflow as tf

# Data Preprocessing :

In [8]:
num_classes = 10
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28,28,1)

# convert class vector to binary class matrices

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255
print(x_train.shape)
print(x_train.shape[0])
print(x_test.shape[0])

(60000, 28, 28, 1)
60000
10000


# Model Creation :

In [22]:
batch_size = 128
num_classes = 10
epochs = 10
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adadelta(),metrics =['accuracy'])


# Training the model :

In [14]:
hist = model.fit(x_train,y_train,batch_size=batch_size,epochs=5,verbose=1,validation_data=(x_test,y_test))
print("The model has successfully trained")
model.save("minist.h5")
print("Saving the model as minist.h5")

Epoch 1/5
469/469 [==============================] - 194s 413ms/step - loss: 2.0457 - accuracy: 0.4979 - val_loss: 1.9436 - val_accuracy: 0.6954
Epoch 2/5
469/469 [==============================] - 196s 417ms/step - loss: 1.8993 - accuracy: 0.5582 - val_loss: 1.7576 - val_accuracy: 0.7276
Epoch 3/5
469/469 [==============================] - 193s 412ms/step - loss: 1.7153 - accuracy: 0.6037 - val_loss: 1.5290 - val_accuracy: 0.7553
Epoch 4/5
469/469 [==============================] - 193s 412ms/step - loss: 1.5028 - accuracy: 0.6414 - val_loss: 1.2849 - val_accuracy: 0.7808
Epoch 5/5
469/469 [==============================] - 195s 417ms/step - loss: 1.3031 - accuracy: 0.6696 - val_loss: 1.0686 - val_accuracy: 0.8027
The model has successfully trained
Saving the model as minist.h5


In [16]:
score = model.evaluate(x_test,y_test,verbose=0)
print('Test loss:', score[0])
print('Test accuracy: ', score[1])

Test loss: 1.0686463117599487
Test accuracy:  0.8026999831199646


# Gui Creation :

In [24]:
from tensorflow.keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np


In [34]:
model = load_model('minist.h5')
def predict_digit(img):
    #resize of image to 28*28 pixels
    img= img.resize((28,28))
    img= img.convert('L')
    img=np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x=self.y=0
        #creating the elements
        self.canvas = tk.Canvas(self, width=300,height=300,bg="white",cursor="cross")
        self.label = tk.Label(self, text = " Trying to Understand !",font=("Helvetica",12))
        self.classify_button = tk.Button(self, text = 'Recognise', command = self.classify_handwriting)
        self.button_clear = tk.Button(self, text = "CLEAR", command = self.clear_all)
        #Grid structures
        self.canvas.grid(row=0,column=0,pady=2,sticky=W,)
        self.label.grid(row=0,column=1,pady=2,padx=2)
        self.classify_button.grid(row=1,column=1,pady=2,padx=2)
        self.button_clear.grid(row=1,column=0,pady=2)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()
        rect= win32gui.GetWindowRect(HWND)
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text = str(digit) + ', '+str(int(acc*100))+"%")
    def draw_lines(self, event):
        self.x=event.x
        self.y=event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r,fill='black')
app = App()
mainloop()